*DATA PIPELINE*

In [ ]:
#Import Libraries
import sklearn
import luigi
import pandas as pd
import csv


In [ ]:
# Set Configuration
data_loc = "MURA"

In [ ]:
class Load_Meta_Data(luigi.Task):
    
    def read_data_from_csv(self,test_or_valid):
        data = []
        with open(data_loc+"/{}_image_paths.csv".format(test_or_valid),"r") as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                path = row[0]
                fields = path.split("/")
                area = fields[2].split("_")[1]
                patient = fields[3].split("patient")[1]
                num = fields[5].replace("image",'').replace(".png",'')
                target = 1 if fields[4].split("_")[1] == "positive" else 0
                data.append([path,area,patient,num,target])

        meta_data = pd.DataFrame(data,columns = ["path", "area", "patient","num","target"])
        return meta_data
 
    def requires(self):
        return []
 
    def output(self):
        return luigi.LocalTarget("pipeline/meta_data.csv")
 
    def run(self):
        test_data = self.read_data_from_csv("train")
        valid_data =  self.read_data_from_csv("valid")
        all_data = pd.concat([test_data,valid_data])
        with self.output().open('w') as file:
            all_data.to_csv(file, index=False)
    
    

In [ ]:
luigi.run(main_task_cls=Load_Meta_Data,cmdline_args=["--local-scheduler"])